In [ ]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from IPython.display import display


In [ ]:
%load_ext autoreload
%autoreload 2
    
from simulation_tools import Population
from simulation_tools import Transformation
from simulation_tools import Metric
from simulation_tools import Experiment


In [ ]:
# Load data:
SYNTHETIC_DATA = './synthetic_data/'

dummy = False

if dummy:

    location_attributes = pd.read_csv(SYNTHETIC_DATA+'location_attributes_dummy.csv')
    location_profiles = pd.read_csv(SYNTHETIC_DATA+'location_profiles_dummy.csv')

else:
    
    location_attributes = pd.read_csv(SYNTHETIC_DATA+'location_attributes_census.csv')
    location_attributes['location_name'] = location_attributes['location_id']

    location_profiles = pd.read_csv(SYNTHETIC_DATA+'location_profiles_census.csv')
    location_profiles['location_name'] = location_profiles['location_id']
    
    # Limit to one state and divide population by a constant to get reasonable sample size.
    state = 'New York'
    constant = 1000
    location_attributes = location_attributes[location_attributes['location_state']==state]
    location_profiles = location_profiles[location_profiles['location_state']==state]
    location_attributes['population'] = (location_attributes['population']/constant).astype(int)
    
    # Rescale standard deviations:
    location_profiles['worktravel_std'] = np.round(location_profiles['worktravel_std']/20,1)
    location_profiles['socialtravel_std'] = np.round(location_profiles['socialtravel_std']/20,1)
    location_profiles['grocerytravel_std'] = np.round(location_profiles['grocerytravel_std']/20,1)
    
print("Loaded data for {} locations ({} people).".format(
    len(location_attributes),location_attributes['population'].sum()
))

display(location_attributes.head())
display(location_profiles.head())


In [ ]:
#population = Population(location_attributes, location_profiles, random_state=221)
#transformation = Transformation(population, behavior='total_compliance', random_state=221)
#metric = Metric(population, transformation, method='median_person', random_state=221)

experiment1 = Experiment(
    location_attributes, location_profiles, behavior='total_compliance',
    metric_method='median_person', summary_method='worst5_status',
    trials=10, show_progress=True, experiment_name=None, random_state=221,
)


In [ ]:
# Raw metrics:
display(experiment1.get_results(style='df').head())
display(experiment1.get_results(style='mean').head())
display(experiment1.get_results(style='std').head())


In [ ]:
# Summarized metrics:
display(experiment1.get_summaries(style='df').head())
display(experiment1.get_summaries(style='mean').head())
display(experiment1.get_summaries(style='std').head())


In [ ]:
# Get results from frist trial:
df = experiment1.get_results(style='df')
df.loc[(experiment1.experiment_name,"trial1")]


In [ ]:
# Get people table from first trial:
experiment1._before[0]
